In [4]:
import pandas as pd
from datasets import load_dataset
from PIL import Image
import numpy as np
import time
import openai
import os

In [23]:

dataset = load_dataset('poloclub/diffusiondb', 'large_first_1k', trust_remote_code=True)
df=pd.DataFrame(dataset)
df.head()

,train
0,{'image': <PIL.WebPImagePlugin.WebPImageFile i...
1,{'image': <PIL.WebPImagePlugin.WebPImageFile i...
2,{'image': <PIL.WebPImagePlugin.WebPImageFile i...
3,{'image': <PIL.WebPImagePlugin.WebPImageFile i...
4,{'image': <PIL.WebPImagePlugin.WebPImageFile i...


In [27]:

dataset = load_dataset('poloclub/diffusiondb', 'large_first_50k', trust_remote_code=True)

In [33]:

batch_size = 10000
df_list = []  # List to store DataFrame batches

# Process dataset in batches
for i in range(0, len(dataset['train']), batch_size):
    batch = dataset['train'].select(range(i, min(i + batch_size, len(dataset['train']))))
    df_batch = batch.to_pandas()
    df_list.append(df_batch)  # Append batch to list

# Concatenate all batches into a final DataFrame
df = pd.concat(df_list, ignore_index=True)
df.info()
# Display the first few rows
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   image        50000 non-null  object             
 1   prompt       50000 non-null  object             
 2   seed         50000 non-null  uint32             
 3   step         50000 non-null  uint16             
 4   cfg          50000 non-null  float32            
 5   sampler      50000 non-null  object             
 6   width        50000 non-null  uint16             
 7   height       50000 non-null  uint16             
 8   user_name    50000 non-null  object             
 9   timestamp    50000 non-null  datetime64[us, UTC]
 10  image_nsfw   50000 non-null  float32            
 11  prompt_nsfw  50000 non-null  float32            
dtypes: datetime64[us, UTC](1), float32(3), object(4), uint16(3), uint32(1)
memory usage: 3.0+ MB


,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw
0,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",911959960,50,10.0,k_lms,704,512,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.055936,0.007284
1,"{'bytes': None, 'path': 'C:\Users\georg\.cache...","goddess portrait, ismail inceoglu",3614153847,50,10.0,k_lms,704,512,d4393f4711a09b71abe9ac39a849b68d70f88ad15b94b0...,2022-08-20 09:32:00+00:00,0.103962,0.007284
2,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",portrait of king of candy mr harry haribo oil ...,2271108630,50,7.0,k_lms,512,512,7c375c633f3ac3961453837abde94580b043b1c2eb5abc...,2022-08-20 09:33:00+00:00,0.034399,0.022897
3,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1947974658,50,7.0,k_lms,832,1024,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.113144,0.001688
4,"{'bytes': None, 'path': 'C:\Users\georg\.cache...",super epic realistic nature photo trending on ...,1035761379,50,7.0,k_lms,832,1024,bd227929e003e69eee392c1d8a409445dd790455c1e9bd...,2022-08-20 09:34:00+00:00,0.096025,0.001688


In [36]:
image_path = df.loc[0, 'image']['path']  # Extract the image path
image = Image.open(image_path)
image.show()
image_array = np.array(image)
print(image_array.shape)

(512, 704, 3)


In [37]:
image_rgb = image.convert('RGB')
image_array = np.expand_dims(image_rgb, axis=0)


In [38]:
print(image_array)

[[[[130 154 160]
   [130 153 163]
   [128 154 164]
   ...
   [129 142 160]
   [129 143 163]
   [132 144 158]]

  [[127 152 163]
   [129 154 166]
   [128 153 165]
   ...
   [128 141 162]
   [127 140 163]
   [128 140 164]]

  [[127 152 166]
   [127 153 166]
   [128 155 167]
   ...
   [129 141 162]
   [130 142 163]
   [130 142 162]]

  ...

  [[115  99 120]
   [116  97 123]
   [117  99 125]
   ...
   [123  96 121]
   [124  98 123]
   [122  97 119]]

  [[115  99 121]
   [117  98 124]
   [116  96 123]
   ...
   [123  94 121]
   [122  97 120]
   [124 101 119]]

  [[112  98 116]
   [116  98 123]
   [115  98 124]
   ...
   [121  99 122]
   [118  96 119]
   [121 101 117]]]]


In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
import getpass
if openai.api_key is None:
    print(' ValueError("API key is missing. Please set the OPENAI_API_KEY environment variable.")')
    openai.api_key = getpass.getpass("Enter your API key: ")
# Initialize the OpenAI client
client = openai

In [40]:
def classify_word(word):
    # Define a system message and user prompt to classify each word
    prompt = f"Classify the word '{word}' into one of the following categories: 'illegal', 'explicit', 'violent', 'innocent'. Provide just the category as the output."

    # Call the OpenAI API with the prompt
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies words."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        
        # Parse the response and extract the classification
        classification = response.choices[0].message.content.strip()
        if 'illegal' in classification:
            classification = 'illegal'
        elif 'explicit' in classification:
            classification = 'explicit'
        elif 'violent' in classification:
            classification = 'violent'
        elif 'innocent' in classification:
            classification = 'innocent'
        # Map the response to a unique value
        category_mapping = {
            "illegal": 1,
            "explicit": 2,
            "violent": 3,
            "innocent": 0
        }
        
        # Return the mapped value
        return category_mapping.get(classification, -1)  # Return -1 for unknown categories
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1

In [41]:
def numberify(word):
    # Define a system message and user prompt to classify each word
    prompt = f"""Assign the word a number based on the following criteria. The number will be 8 digits. the first 4 are to determine how much censorship the word may be. 
    Ensure a pattern is involved. Filler words will be 1111xxxx. Subjects(non explicit) will be 2222xxxx. Verbs (non explicit) will be 3333xxxx. Then have forward scaling for explicit words. 
    For example the word 'sex' will be 8888xxxx and the word 'rape' will be 9999xxxx. Have intermediate words follow a similar scale. 
    The final four numbers will be determined with the following. The first 2 numbers of the final 4 numbers will be based on the amount of letters in the word. The following 2 numbers will be a fine scale of possible explicit value. 
    For example, the word sex will be classified as 88880395. If a word could have potential implications to violence, illegality, drugs, or explicit content, then give it a higher number than 400000. Use a good scale though. words like assassin or shot shouldnt be classified as 9999xxxx. Try to keep a balance with 5555xxxx, 6666xxxx, 7777xxxx, 8888xxxx with 9999xxxx being used to depict severe content like rape, incest, child porn, etc..
    Respond with just the number! HERE IS THE WORD: {word}"""

    # Call the OpenAI API with the prompt
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies words."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        
        # Parse the response and extract the classification
        classification = response.choices[0].message.content.strip()        
        worded = word
        return classification, worded
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   image        50000 non-null  object             
 1   prompt       50000 non-null  object             
 2   seed         50000 non-null  uint32             
 3   step         50000 non-null  uint16             
 4   cfg          50000 non-null  float32            
 5   sampler      50000 non-null  object             
 6   width        50000 non-null  uint16             
 7   height       50000 non-null  uint16             
 8   user_name    50000 non-null  object             
 9   timestamp    50000 non-null  datetime64[us, UTC]
 10  image_nsfw   50000 non-null  float32            
 11  prompt_nsfw  50000 non-null  float32            
dtypes: datetime64[us, UTC](1), float32(3), object(4), uint16(3), uint32(1)
memory usage: 3.0+ MB


In [43]:
import re

# Example DataFrame (replace this with your actual DataFrame)

word_classifications = {}
rows = []
words_set = set()
# Loop through the DataFrame and classify words across all rows
for idx, row in df.iterrows():
    prompt = row['prompt']
    for word in re.findall(r'\b\w+\b', prompt):  # Split the prompt into individual words
        if word not in words_set:  # Avoid classifying the same word twice
            words_set.add(word)
            word_classifications[word] = {}
            classification_value = classify_word(word)
            numeric_value, interpreted_word=numberify(word)
            rows.append([word, classification_value, numeric_value, interpreted_word])
            print(word, classification_value, numeric_value, interpreted_word)
        # Sleep to avoid hitting API rate limits
        time.sleep(.5)  # Adjust the sleep time as necessary

# Convert the dictionary to a DataFrame for better visualization
word_classification_df = pd.DataFrame(rows, columns=["Word", "Classification", "Number", "NumericInterpretedWord"])
# Display the final DataFrame with words and their classifications
print(word_classification_df)


goddess 0 33330300 goddess
portrait 0 22220400 portrait
ismail 0 22220400 ismail
inceoglu 0 22220400 inceoglu
of 0 11110000 of
king 0 22220100 king
candy 0 22220401 candy
mr 0 11110000 mr
harry 0 22220400 harry
haribo 0 22220056 haribo
oil 0 22220000 oil
painting 0 22220400 painting
bloody 3 66660375 bloody
conquest 3 33332105 conquest
tap 0 33330201 tap
e 0 11110000 e
super 0 22220004 super
epic 0 22221010 epic
realistic 0 22220300 realistic
nature 0 22220012 nature
photo 0 22220400 photo
trending 0 22220404 trending
on 0 11110000 on
instagram 0 22220301 instagram
with 0 11110000 with
lonely 0 22220405 lonely
person 0 22220000 person
in 0 11110000 in
yellow 0 22220100 yellow
raincoat 0 22220301 raincoat
standing 0 33330200 standing
at 0 11110000 at
a 0 11110000 a
distance 0 22220400 distance
beautiful 0 22220100 beautiful
princess 0 22220400 princess
wearing 0 33330100 wearing
evil -1 22220405 evil
black 0 22220400 black
oily 0 22220401 oily
tar 0 22220100 tar
by 0 11110000 by
hr 0 11

TypeError: cannot unpack non-iterable int object

In [ ]:
print(word_classification_df['Classification'].unique())